In [1]:
import pandas as pd
import numpy as np
from concurrent.futures import ThreadPoolExecutor, as_completed
from typing import Union, Dict, List

from pathlib import Path

import seaborn as sns
import matplotlib.pyplot as plt

## Begin Data Prep (not for learners)

In [2]:
ls

Module3_NC_dataEng.ipynb   hourly_data_MITC_2024.csv
hourly_data_BEAR_2017.csv  hourly_data_MORG_2024.csv
hourly_data_BEAR_2018.csv  hourly_data_NCAT_2017.csv
hourly_data_BEAR_2019.csv  hourly_data_NCAT_2018.csv
hourly_data_BEAR_2020.csv  hourly_data_NCAT_2019.csv
hourly_data_BEAR_2021.csv  hourly_data_NCAT_2020.csv
hourly_data_BEAR_2022.csv  hourly_data_NCAT_2021.csv
hourly_data_BEAR_2023.csv  hourly_data_NCAT_2022.csv
hourly_data_BEAR_2024.csv  hourly_data_NCAT_2023.csv
hourly_data_BURN_2017.csv  hourly_data_NCAT_2024.csv
hourly_data_BURN_2018.csv  hourly_data_SALI_2017.csv
hourly_data_BURN_2019.csv  hourly_data_SALI_2018.csv
hourly_data_BURN_2020.csv  hourly_data_SALI_2019.csv
hourly_data_BURN_2021.csv  hourly_data_SALI_2020.csv
hourly_data_BURN_2022.csv  hourly_data_SALI_2021.csv
hourly_data_BURN_2023.csv  hourly_data_SALI_2022.csv
hourly_data_BURN_2024.csv  hourly_data_SALI_2023.csv
hourly_data_FRYI_2017.csv  hourly_data_SALI_2024.csv
hourly_data_FRYI_2018.csv  hourly_data_SASS_20

In [3]:
def combine_csvs(folder_path, concat_axis=0):
    """
    Load and combine all CSV files from a folder into a single DataFrame.
    Checks for column name consistency and strips headers from subsequent files.
    """
    path = Path(folder_path)
    dataframes = []
    first_columns = None
    file_sources = []  # Keep track of source files and their row counts
    
    for csv_file in path.glob('*.csv'):
        try:
            df = pd.read_csv(csv_file)
            
            # Clean column names (strip whitespace)
            df.columns = df.columns.str.strip()
            
            # For the first file, store the column names
            if first_columns is None:
                first_columns = set(df.columns)
                dataframes.append(df)
            else:
                # Check if current file has the same columns
                current_columns = set(df.columns)
                if current_columns != first_columns:
                    print(f"Warning: {csv_file.name} has different columns:")
                    print(f"Missing columns: {first_columns - current_columns}")
                    print(f"Extra columns: {current_columns - first_columns}")
                    print("Skipping this file...")
                    continue
                
                dataframes.append(df)
            
            # Store the source file information with correct row count
            file_sources.extend([csv_file.name] * len(df))
            
        except Exception as e:
            print(f"Error reading {csv_file.name}: {e}")
    
    if not dataframes:
        raise ValueError("No CSV files were successfully loaded")
    
    # Combine all dataframes
    combined_df = pd.concat(dataframes, axis=concat_axis, ignore_index=True)
    
    # Add source file information using our tracked sources
    combined_df['source_file'] = file_sources
    
    return combined_df

In [4]:
try:
    df = combine_csvs(".")
    print(f"Successfully combined {len(df)} rows")
except ValueError as e:
    print(f"Error: {e}")

Successfully combined 670927 rows


In [5]:
# Dropping MORG 
df = df[df['location_id']!='MORG']

In [6]:
df.dtypes

location_id              object
observation_datetime     object
observation_timezone     object
airtemp_degF            float64
windspeed_avg_mph       float64
winddgust_mph           float64
rhavg_percent           float64
precip_in               float64
source_file              object
dtype: object

In [7]:
df.describe()

,airtemp_degF,windspeed_avg_mph,winddgust_mph,rhavg_percent,precip_in
count,667398.000000,663464.000000,667605.000000,663960.000000,667403.000000
mean,52.726884,7.489819,10.011427,73.638571,0.007579
std,15.699536,6.215288,8.048214,21.731251,0.047163
min,-136.840000,0.000000,0.000000,0.000000,0.000000
25%,41.900000,3.131800,4.297280,58.400000,0.000000
50%,54.140000,6.039900,8.285850,79.000000,0.000000
75%,64.040000,10.066500,13.636800,92.200000,0.000000
max,349.700000,57.490900,78.272600,100.000000,4.140000


In [8]:
# Check the number of NaN values in each column
print(df.isna().sum())

location_id                0
observation_datetime       0
observation_timezone       0
airtemp_degF             207
windspeed_avg_mph       4141
winddgust_mph              0
rhavg_percent           3645
precip_in                202
source_file                0
dtype: int64


In [9]:
# Calculate the percentage of NaN values in each column
nan_percentage = (df.isna().sum() / len(df)) * 100

# Optionally, format the output to 2 decimal places
print("\nPercentage of NaN values in each column (formatted):")
print(nan_percentage.round(2))


Percentage of NaN values in each column (formatted):
location_id             0.00
observation_datetime    0.00
observation_timezone    0.00
airtemp_degF            0.03
windspeed_avg_mph       0.62
winddgust_mph           0.00
rhavg_percent           0.55
precip_in               0.03
source_file             0.00
dtype: float64


In [10]:
num_unique_stations = df['location_id'].nunique()

print(f"Number of unique stations: {num_unique_stations}")

unique_stations_list = df['location_id'].unique().tolist()

print(f"List of unique stations: {unique_stations_list}")

Number of unique stations: 10
List of unique stations: ['BURN', 'NCAT', 'SALI', 'MITC', 'SASS', 'FRYI', 'JEFF', 'BEAR', 'WINE', 'UNCA']


In [11]:
# Convert 'date' column to datetime
df['date'] = pd.to_datetime(df['observation_datetime'])

# Find the lowest date
lowest_date = df['date'].min()

# Create a new column 'day_index' starting from 0
df['day_index'] = (df['date'] - lowest_date).dt.days

# Create a new column 'hour_index' going from 0 to 23
df['hour_index'] = df['date'].dt.hour

# Create a new column 'year_index' starting from 0
df['year_index'] = df['date'].dt.year - df['date'].dt.year.min()

df

,location_id,observation_datetime,observation_timezone,airtemp_degF,windspeed_avg_mph,winddgust_mph,rhavg_percent,precip_in,source_file,date,day_index,hour_index,year_index
0,BURN,2018-01-01 00:00:00,EST,13.1000,5.8162,9.60120,69.1,0.0,hourly_data_BURN_2018.csv,2018-01-01 00:00:00,365,0,1
1,BURN,2018-01-01 01:00:00,EST,12.3800,4.9214,6.70876,69.4,0.0,hourly_data_BURN_2018.csv,2018-01-01 01:00:00,365,1,1
2,BURN,2018-01-01 02:00:00,EST,10.9400,4.9214,5.69988,71.3,0.0,hourly_data_BURN_2018.csv,2018-01-01 02:00:00,365,2,1
3,BURN,2018-01-01 03:00:00,EST,10.0400,3.3555,6.05108,72.5,0.0,hourly_data_BURN_2018.csv,2018-01-01 03:00:00,365,3,1
4,BURN,2018-01-01 04:00:00,EST,9.3200,3.5792,5.21668,72.7,0.0,hourly_data_BURN_2018.csv,2018-01-01 04:00:00,365,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
670922,UNCA,2019-12-31 19:00:00,EST,38.3684,0.4474,4.23911,53.6,0.0,hourly_data_UNCA_2019.csv,2019-12-31 19:00:00,1094,19,2
670923,UNCA,2019-12-31 20:00:00,EST,37.1480,1.3422,4.67757,59.9,0.0,hourly_data_UNCA_2019.csv,2019-12-31 20:00:00,1094,20,2
670924,UNCA,2019-12-31 21:00:00,EST,37.3298,2.2370,3.28839,61.1,0.0,hourly_data_UNCA_2019.csv,2019-12-31 21:00:00,1094,21,2
670925,UNCA,2019-12-31 22:00:00,EST,36.4136,4.0266,6.50296,63.2,0.0,hourly_data_UNCA_2019.csv,2019-12-31 22:00:00,1094,22,2


In [12]:
df.columns

Index(['location_id', 'observation_datetime', 'observation_timezone',
       'airtemp_degF', 'windspeed_avg_mph', 'winddgust_mph', 'rhavg_percent',
       'precip_in', 'source_file', 'date', 'day_index', 'hour_index',
       'year_index'],
      dtype='object')

In [13]:
def process_station_data(df, station):
    print(f"Processing station: {station}")
    print(f"Shape of df before filtering: {df.shape}")
    
    # Filter the DataFrame for the specific station
    df_station = df[df['location_id'] == station].copy()
    
    print(f"Shape of df_station after filtering: {df_station.shape}")
    
    if df_station.empty:
        print(f"No data found for station {station}")
        return None
    
    # Drop columns we don't need
    columns_to_drop = ['location_id', 'source_file', 'observation_timezone']
    df_station = df_station.drop(columns=[col for col in columns_to_drop if col in df_station.columns])
    
    # Define columns we want to keep unchanged (no station suffix)
    unchanged_cols = [
        'date', 
        'day_index', 
        'hour_index', 
        'year_index',
        'observation_datetime'
    ]
    
    # Create mapping for measurement columns to their new names
    measurement_renames = {
        'airtemp_degF': f'{station}_airtemp_degF',
        'windspeed_avg_mph': f'{station}_windspeed_mph',
        'winddgust_mph': f'{station}_windgust_mph',
        'rhavg_percent': f'{station}_rh_percent',
        'precip_in': f'{station}_precip_in'
    }
    
    # Apply the measurement-specific renames
    df_station = df_station.rename(columns=measurement_renames)
    
    return df_station

In [14]:
# Debugging: Print information about df and unique_stations_list
print(f"Shape of df: {df.shape}")
print(f"Columns in df: {df.columns}")
print(f"Unique values in 'Station' column: {df['location_id'].unique()}")
print(f"unique_stations_list: {unique_stations_list}")

Shape of df: (667605, 13)
Columns in df: Index(['location_id', 'observation_datetime', 'observation_timezone',
       'airtemp_degF', 'windspeed_avg_mph', 'winddgust_mph', 'rhavg_percent',
       'precip_in', 'source_file', 'date', 'day_index', 'hour_index',
       'year_index'],
      dtype='object')
Unique values in 'Station' column: ['BURN' 'NCAT' 'SALI' 'MITC' 'SASS' 'FRYI' 'JEFF' 'BEAR' 'WINE' 'UNCA']
unique_stations_list: ['BURN', 'NCAT', 'SALI', 'MITC', 'SASS', 'FRYI', 'JEFF', 'BEAR', 'WINE', 'UNCA']


In [15]:
# Initialize an empty DataFrame to store the merged results
merged_df = pd.DataFrame()

# Define columns that should not be renamed (merge keys)
unchanged_cols = ['date', 'day_index', 'hour_index', 'year_index', 'observation_datetime']

# Iterate through each station in the list of unique stations
for station in unique_stations_list:
    # Process the data for the current station
    processed_df = process_station_data(df, station)
    
    if processed_df is not None and not processed_df.empty:
        # Note: Removed the rename step since it's handled in process_station_data
        
        if merged_df.empty:
            merged_df = processed_df
        else:
            merged_df = pd.merge(merged_df, processed_df, 
                               on=unchanged_cols, 
                               how='outer')
        
        print(f"Station {station} processed successfully!")
    else:
        print(f"Skipping empty or None result for station {station}")

Processing station: BURN
Shape of df before filtering: (667605, 13)
Shape of df_station after filtering: (68596, 13)
Station BURN processed successfully!
Processing station: NCAT
Shape of df before filtering: (667605, 13)
Shape of df_station after filtering: (69623, 13)
Station NCAT processed successfully!
Processing station: SALI
Shape of df before filtering: (667605, 13)
Shape of df_station after filtering: (69496, 13)
Station SALI processed successfully!
Processing station: MITC
Shape of df before filtering: (667605, 13)
Shape of df_station after filtering: (67979, 13)
Station MITC processed successfully!
Processing station: SASS
Shape of df before filtering: (667605, 13)
Shape of df_station after filtering: (69066, 13)
Station SASS processed successfully!
Processing station: FRYI
Shape of df before filtering: (667605, 13)
Shape of df_station after filtering: (69282, 13)
Station FRYI processed successfully!
Processing station: JEFF
Shape of df before filtering: (667605, 13)
Shape of

In [16]:
merged_df.columns

Index(['observation_datetime', 'BURN_airtemp_degF', 'BURN_windspeed_mph',
       'BURN_windgust_mph', 'BURN_rh_percent', 'BURN_precip_in', 'date',
       'day_index', 'hour_index', 'year_index', 'NCAT_airtemp_degF',
       'NCAT_windspeed_mph', 'NCAT_windgust_mph', 'NCAT_rh_percent',
       'NCAT_precip_in', 'SALI_airtemp_degF', 'SALI_windspeed_mph',
       'SALI_windgust_mph', 'SALI_rh_percent', 'SALI_precip_in',
       'MITC_airtemp_degF', 'MITC_windspeed_mph', 'MITC_windgust_mph',
       'MITC_rh_percent', 'MITC_precip_in', 'SASS_airtemp_degF',
       'SASS_windspeed_mph', 'SASS_windgust_mph', 'SASS_rh_percent',
       'SASS_precip_in', 'FRYI_airtemp_degF', 'FRYI_windspeed_mph',
       'FRYI_windgust_mph', 'FRYI_rh_percent', 'FRYI_precip_in',
       'JEFF_airtemp_degF', 'JEFF_windspeed_mph', 'JEFF_windgust_mph',
       'JEFF_rh_percent', 'JEFF_precip_in', 'BEAR_airtemp_degF',
       'BEAR_windspeed_mph', 'BEAR_windgust_mph', 'BEAR_rh_percent',
       'BEAR_precip_in', 'WINE_airtemp_

In [17]:
merged_df.describe()

,BURN_airtemp_degF,BURN_windspeed_mph,BURN_windgust_mph,BURN_rh_percent,BURN_precip_in,date,day_index,hour_index,year_index,NCAT_airtemp_degF,...,WINE_airtemp_degF,WINE_windspeed_mph,WINE_windgust_mph,WINE_rh_percent,WINE_precip_in,UNCA_airtemp_degF,UNCA_windspeed_mph,UNCA_windgust_mph,UNCA_rh_percent,UNCA_precip_in
count,68596.000000,68363.000000,68596.000000,68336.000000,68596.000000,69760,69760.00000,69760.000000,69760.000000,69623.000000,...,68501.000000,68440.000000,68510.000000,68413.000000,68510.000000,46283.000000,46230.000000,46283.000000,46266.000000,46283.000000
mean,54.372067,4.308732,5.981867,71.871230,0.006658,2020-12-24 13:38:27.591743232,1453.08922,11.499713,3.483214,60.237652,...,48.570732,6.108281,9.460373,76.972752,0.007932,56.637852,3.391849,5.165646,72.357061,0.004006
min,-4.180000,0.000000,0.000000,3.100000,0.000000,2017-01-01 00:00:00,0.00000,0.000000,0.000000,4.460000,...,-14.980000,0.000000,0.000000,0.000000,0.000000,-2.740000,0.000000,0.000000,1.100000,0.000000
25%,42.620000,1.118500,1.709070,57.200000,0.000000,2018-12-28 21:45:00,726.00000,5.000000,1.000000,48.020000,...,39.380000,3.355500,5.393410,62.900000,0.000000,45.603500,1.342200,1.899210,57.000000,0.000000
50%,56.480000,4.026600,5.306160,75.300000,0.000000,2020-12-24 15:30:00,1453.00000,11.000000,3.000000,62.060000,...,50.360000,5.592500,8.637060,86.700000,0.000000,58.496000,2.684400,4.019890,76.500000,0.000000
75%,66.200000,6.487300,9.120250,89.400000,0.000000,2022-12-21 07:15:00,2180.00000,17.000000,5.000000,72.500000,...,59.360000,8.276900,12.715100,94.300000,0.000000,67.820000,4.697700,7.379860,91.700000,0.000000
max,90.140000,23.488500,41.049000,100.000000,2.160000,2024-12-16 23:00:00,2906.00000,23.000000,7.000000,95.540000,...,129.920000,23.712200,41.787200,100.000000,1.880000,349.700000,17.448600,44.650500,100.000000,2.020000
std,15.607199,3.348480,5.098407,20.420981,0.043201,NaN,839.18155,6.922258,2.283490,16.047654,...,13.799802,3.517905,5.475936,23.558845,0.043443,15.353779,2.647189,4.485656,21.132323,0.032385


#### Splitting data for ML

In [18]:
def impute_with_rolling_average_and_median(df, window_size=5):
    df_imputed = df.copy()
    
    for column in df_imputed.columns:
        if df_imputed[column].dtype.kind in 'biufc':  # Check if column is numeric
            # Create a Series with the rolling mean
            rolling_mean = df_imputed[column].rolling(window=window_size, center=True, min_periods=1).mean()
            
            # Use the rolling mean to fill NaN values
            df_imputed[column] = df_imputed[column].fillna(rolling_mean)
            
            # If any NaNs remain, fill with the median of the column
            if df_imputed[column].isna().any():
                column_median = df_imputed[column].median()
                df_imputed[column] = df_imputed[column].fillna(column_median)
                print(f"Column '{column}': Filled remaining NaNs with median ({column_median})")
        else:
            print(f"Column '{column}' is non-numeric. Skipping imputation.")
    
    return df_imputed

In [19]:
df_imputed = impute_with_rolling_average_and_median(merged_df)

Column 'observation_datetime' is non-numeric. Skipping imputation.
Column 'BURN_airtemp_degF': Filled remaining NaNs with median (56.48)
Column 'BURN_windspeed_mph': Filled remaining NaNs with median (4.0266)
Column 'BURN_windgust_mph': Filled remaining NaNs with median (5.30616)
Column 'BURN_rh_percent': Filled remaining NaNs with median (75.3)
Column 'BURN_precip_in': Filled remaining NaNs with median (0.0)
Column 'date' is non-numeric. Skipping imputation.
Column 'NCAT_airtemp_degF': Filled remaining NaNs with median (62.06)
Column 'NCAT_windspeed_mph': Filled remaining NaNs with median (4.0266)
Column 'NCAT_windgust_mph': Filled remaining NaNs with median (5.17418)
Column 'NCAT_rh_percent': Filled remaining NaNs with median (70.8)
Column 'NCAT_precip_in': Filled remaining NaNs with median (0.0)
Column 'SALI_airtemp_degF': Filled remaining NaNs with median (62.06)
Column 'SALI_windspeed_mph': Filled remaining NaNs with median (2.6844)
Column 'SALI_windgust_mph': Filled remaining NaN

In [20]:
df_imputed.columns

Index(['observation_datetime', 'BURN_airtemp_degF', 'BURN_windspeed_mph',
       'BURN_windgust_mph', 'BURN_rh_percent', 'BURN_precip_in', 'date',
       'day_index', 'hour_index', 'year_index', 'NCAT_airtemp_degF',
       'NCAT_windspeed_mph', 'NCAT_windgust_mph', 'NCAT_rh_percent',
       'NCAT_precip_in', 'SALI_airtemp_degF', 'SALI_windspeed_mph',
       'SALI_windgust_mph', 'SALI_rh_percent', 'SALI_precip_in',
       'MITC_airtemp_degF', 'MITC_windspeed_mph', 'MITC_windgust_mph',
       'MITC_rh_percent', 'MITC_precip_in', 'SASS_airtemp_degF',
       'SASS_windspeed_mph', 'SASS_windgust_mph', 'SASS_rh_percent',
       'SASS_precip_in', 'FRYI_airtemp_degF', 'FRYI_windspeed_mph',
       'FRYI_windgust_mph', 'FRYI_rh_percent', 'FRYI_precip_in',
       'JEFF_airtemp_degF', 'JEFF_windspeed_mph', 'JEFF_windgust_mph',
       'JEFF_rh_percent', 'JEFF_precip_in', 'BEAR_airtemp_degF',
       'BEAR_windspeed_mph', 'BEAR_windgust_mph', 'BEAR_rh_percent',
       'BEAR_precip_in', 'WINE_airtemp_

In [21]:
def process_mitc_columns(df, cutoff_date='2024-09-28'):
    # Make a copy to avoid modifying the original DataFrame
    df_processed = df.copy()
    
    # Get list of columns containing 'MITC'
    mitc_columns = [col for col in df.columns if 'MITC' in col]
    print(f"Found MITC columns: {mitc_columns}")
    
    # Create mask for dates after cutoff
    cutoff_dt = pd.to_datetime(cutoff_date)
    after_cutoff_mask = df_processed['date'] > cutoff_dt
    
    # Calculate number of days
    dates_after_cutoff = df_processed.loc[after_cutoff_mask, 'date']
    num_days = len(pd.date_range(dates_after_cutoff.min(), dates_after_cutoff.max()).date)
    
    print(f"Date range in data: {df_processed['date'].min()} to {df_processed['date'].max()}")
    print(f"Cutoff date: {cutoff_dt}")
    print(f"Number of rows after cutoff: {after_cutoff_mask.sum()}")
    print(f"Number of days being NaN'd: {num_days}")
    print(f"Hours being NaN'd: {after_cutoff_mask.sum()}")
    print(f"Average hours per day being NaN'd: {after_cutoff_mask.sum() / num_days:.2f}")
    
    # Set values to NaN for all MITC columns after cutoff date
    for col in mitc_columns:
        df_processed.loc[after_cutoff_mask, col] = np.nan
        
    return df_processed

In [22]:
df_imputed = process_mitc_columns(df_imputed)

Found MITC columns: ['MITC_airtemp_degF', 'MITC_windspeed_mph', 'MITC_windgust_mph', 'MITC_rh_percent', 'MITC_precip_in']
Date range in data: 2017-01-01 00:00:00 to 2024-12-16 23:00:00
Cutoff date: 2024-09-28 00:00:00
Number of rows after cutoff: 1919
Number of days being NaN'd: 80
Hours being NaN'd: 1919
Average hours per day being NaN'd: 23.99


In [23]:
df_imputed.to_parquet('../processed_data/NC_processed_data_1_6.parquet')